# Pipeline para a 3W

Neste notebook será implementado uma Pipiline de ML aplicado ao problema da dataset 3W da Petrobras.

Para tal será usada a biblioteca TensorFlow Extended.

Autoria: Marcus Carr

### Nomenclatura

instance/instância de um **evento**: equivale a 1 arquivo .csv

**sample**: cada timestep dentro de um .csv

### Estrutura do projeto. 

Como ainda não sei como será tudo com a implementação do módulos do TFX, vou deixar um módulo principal por enquanto.

Posteriormente, dá para analisar se seria mais adequado quebrar em diferentes módulos a estrutra do código.

In [5]:
# Verificar que TFX está instalado
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.12.1
TFX version: 1.13.0


### Seting up variables

These will define our pipeline.

In [6]:
import os

PIPELINE_NAME = "pipeline-3w"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# Set default logging level.
from absl import logging
logging.set_verbosity(logging.INFO)

In [7]:
import raw_data_acquisition as rda
import raw_data_inspector as rdi
from constants import utils, config
import models

Adquirir dados!

In [8]:
rda.acquire_dataset_if_needed() # 17min48s

--- Logging error ---
Traceback (most recent call last):
  File "/home/aipas/.pyenv/versions/3.9.17/lib/python3.9/logging/__init__.py", line 1083, in emit
    msg = self.format(record)
  File "/home/aipas/.pyenv/versions/3.9.17/lib/python3.9/logging/__init__.py", line 927, in format
    return fmt.format(record)
  File "/home/aipas/.pyenv/versions/3.9.17/lib/python3.9/logging/__init__.py", line 663, in format
    record.message = record.getMessage()
  File "/home/aipas/.pyenv/versions/3.9.17/lib/python3.9/logging/__init__.py", line 367, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/home/aipas/.pyenv/versions/3.9.17/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/aipas/.pyenv/versions/3.9.17/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/aipas/Documents/coding/lemi_3w/.venv/lib/python3.9/site-packa

TypeError: can only concatenate str (not "int") to str

In [ ]:
print(f"Size of directory with converted files is: {utils.get_directory_size(config.DIR_CONVERTED_DATASET)/(1024**3):.3f} GB")

Size of directory with converted files is: 1.263 GB


In [ ]:
inspector = rdi.RawDataInspector(
    config.DIR_CONVERTED_DATASET,
    config.PATH_DATA_INSPECTOR_CACHE,
    True
)

inspector.get_metadata_table()

,class_type,source,well_id,path,timestamp,file_size,num_timesteps
hash_id,,,,,,,
12eb534,SPURIOUS_CLOSURE_DHSV,REAL,12.0,/home/aipas/Documents/coding/lemi_3w/data/data...,2017-03-20 02:10:00,124392,6939
5cc826a,SPURIOUS_CLOSURE_DHSV,SIMULATED,NaN,/home/aipas/Documents/coding/lemi_3w/data/data...,NaT,945062,28799
c82853f,SPURIOUS_CLOSURE_DHSV,SIMULATED,NaN,/home/aipas/Documents/coding/lemi_3w/data/data...,NaT,935871,28799
9f475ba,SPURIOUS_CLOSURE_DHSV,REAL,9.0,/home/aipas/Documents/coding/lemi_3w/data/data...,2017-03-13 16:08:04,237274,6738
e7cec40,SPURIOUS_CLOSURE_DHSV,REAL,11.0,/home/aipas/Documents/coding/lemi_3w/data/data...,2014-05-15 10:46:09,144840,22409
...,...,...,...,...,...,...,...
f0936fa,QUICK_RESTRICTION_PCK,SIMULATED,NaN,/home/aipas/Documents/coding/lemi_3w/data/data...,NaT,645552,26999
3549e12,QUICK_RESTRICTION_PCK,SIMULATED,NaN,/home/aipas/Documents/coding/lemi_3w/data/data...,NaT,770200,26999
3aeb749,QUICK_RESTRICTION_PCK,SIMULATED,NaN,/home/aipas/Documents/coding/lemi_3w/data/data...,NaT,480640,26999


In [ ]:
inspector.get_metadata_table(class_types=[models.EventClassType(8).name])

,class_type,source,well_id,path,timestamp,file_size,num_timesteps
hash_id,,,,,,,
caf4d77,HYDRATE_IN_PRODUCTION_LINE,SIMULATED,NaN,/home/aipas/Documents/coding/lemi_3w/data/data...,NaT,852114,26999
5b7fe50,HYDRATE_IN_PRODUCTION_LINE,SIMULATED,NaN,/home/aipas/Documents/coding/lemi_3w/data/data...,NaT,842411,26999
4fda0d1,HYDRATE_IN_PRODUCTION_LINE,SIMULATED,NaN,/home/aipas/Documents/coding/lemi_3w/data/data...,NaT,903977,26999
a59a858,HYDRATE_IN_PRODUCTION_LINE,SIMULATED,NaN,/home/aipas/Documents/coding/lemi_3w/data/data...,NaT,947840,26999
5fa48a9,HYDRATE_IN_PRODUCTION_LINE,SIMULATED,NaN,/home/aipas/Documents/coding/lemi_3w/data/data...,NaT,882326,26999
...,...,...,...,...,...,...,...
096aea5,HYDRATE_IN_PRODUCTION_LINE,SIMULATED,NaN,/home/aipas/Documents/coding/lemi_3w/data/data...,NaT,870658,26999
5c33f90,HYDRATE_IN_PRODUCTION_LINE,SIMULATED,NaN,/home/aipas/Documents/coding/lemi_3w/data/data...,NaT,872063,26999
237d812,HYDRATE_IN_PRODUCTION_LINE,SIMULATED,NaN,/home/aipas/Documents/coding/lemi_3w/data/data...,NaT,890828,26999


In [13]:
from pathlib import Path
from constants import config


#examples_uri = config.SCHEMA_PIPELINE_ROOT / "FileBasedExampleGen/examples/5/Split-train"
tfrecord_filenames = Path('data/pipeline/pipeline_lemi_3w_schema/FileBasedExampleGen/examples/5/Split-train/data_tfrecord-00000-of-00001.gz')
#tfrecord_filenames = [examples_uri / name for name in os.listdir(os.path.join(examples_uri, 'train'))]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
for tfrecord in dataset.take(2):
  example = tf.train.Example()
  example.ParseFromString(tfrecord.numpy())
  for name, feature in example.features.feature.items():
    if feature.HasField('bytes_list'):
        value = feature.bytes_list.value
    if feature.HasField('float_list'):
        value = feature.float_list.value
    if feature.HasField('int64_list'):
        value = feature.int64_list.value
    print('{}: {}'.format(name, value))
  print('******')

2023-08-18 09:53:48.824396: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-18 09:53:48.826661: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-08-18 09:53:49.015745: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Pla

NotFoundError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} data/pipeline/pipeline_lemi_3w_schema/FileBasedExampleGen/examples/5/Split-train/data_tfrecord-00000-of-00001.gz; No such file or directory [Op:IteratorGetNext]

In [9]:
tfrecord_filenames

NameError: name 'tfrecord_filenames' is not defined